# Multistage Stochastic Unit Commitment Problem

In [103]:
import os
import numpy as np
import pandas as pd
import gurobipy as gp
from scipy import stats, linalg
from itertools import product


from sddip import config, utils


## Parameters

In [104]:
test_case_raw_dir = "WB2/raw"

test_case_raw_dir = os.path.join(config.test_cases_dir, test_case_raw_dir)

bus_file_raw = os.path.join(test_case_raw_dir, "bus_data.txt")
branch_file_raw = os.path.join(test_case_raw_dir, "branch_data.txt")
gen_file_raw = os.path.join(test_case_raw_dir, "gen_data.txt")
gen_cost_file_raw = os.path.join(test_case_raw_dir, "gen_cost_data.txt")
scenario_data_file = os.path.join(test_case_raw_dir, "scenario_data.txt")

bus_df = pd.read_csv(bus_file_raw, delimiter="\s+")
branch_df = pd.read_csv(branch_file_raw, delimiter="\s+")
gen_df = pd.read_csv(gen_file_raw, delimiter="\s+")
gen_cost_df = pd.read_csv(gen_cost_file_raw, delimiter="\s+")

scenario_df = pd.read_csv(scenario_data_file, delimiter="\s+")

In [105]:
nodes = bus_df.bus_i.values.tolist()
edges = branch_df[["fbus", "tbus"]].values.tolist()

graph = utils.Graph(nodes, edges)

ref_bus = bus_df.loc[bus_df.type == 3].bus_i.values[0]

a_inc = graph.incidence_matrix()
b_l = (-branch_df.x /(branch_df.r**2 + branch_df.x**2)).tolist()
b_diag = np.diag(b_l)

m1 = b_diag.dot(a_inc)
m2 = a_inc.T.dot(b_diag).dot(a_inc)

m1 = np.delete(m1, ref_bus-1, 1)
m2 = np.delete(m2, ref_bus-1, 0)
m2 = np.delete(m2, ref_bus-1, 1)

ptdf = m1.dot(np.linalg.inv(m2))

ptdf = np.insert(ptdf, ref_bus-1, 0, axis=1)

In [106]:
########################################################################################################################
# Deterministic parameters
########################################################################################################################
gc = np.array(gen_cost_df.c1)
suc = np.array(gen_cost_df.startup)
sdc = np.array( gen_cost_df.startup)
pg_min = np.array(gen_df.Pmin)
pg_max = np.array(gen_df.Pmax)
pl_max = np.array(branch_df.rateA)

n_gens = len(gc)
n_lines, n_buses = ptdf.shape

# Lists of generators at each bus
#
# Example: [[0,1], [], [2]]
# Generator 1 & 2 are located at bus 1
# No Generator is located at bus 2
# Generator 3 is located at bus 3
gens_at_bus = [[] for _ in range(n_buses)]
g = 0
for b in gen_df.bus.values:
    gens_at_bus[b-1].append(g)
    g+=1

# TODO Add ramp rate limits
rg_up_max = np.full(n_gens, 1000)
rg_down_max = np.full(n_gens, 1000)

# TODO Adjust penalty
penalty = 10000
    
########################################################################################################################
# Stochastic parameters
########################################################################################################################
n_nodes_per_stage = scenario_df.groupby("t")["n"].nunique().tolist()
n_stages = len(n_nodes_per_stage)

prob = []
p_d = []

for t in range(n_stages):
    stage_df = scenario_df[scenario_df["t"] == t+1]
    p_d.append(stage_df[scenario_df.columns[scenario_df.columns.to_series().str.contains('Pd')]].values.tolist())
    prob.append(stage_df["p"].values.tolist())


########################################################################################################################
# Expected values of stochastic parameters
########################################################################################################################
ex_pd = [np.array(prob[t]).dot(np.array(p_d[t])) for t in range(n_stages)]

In [107]:
# TODO min up-/down-time
min_up_time = [2]*n_gens
min_down_time = [2]*n_gens

In [108]:
gens_at_bus

[[0], []]

In [109]:
# prob[t][n]
# Probability of realization n at stage t
#prob

In [110]:
# p_d[t][n][b]
# Demand in stage t and realization n at bus b
#p_d

In [111]:
# ex_pd[t][b]
# Expected demand in stage t at bus b
#ex_pd

## Optimization

### Variables

In [112]:
model = gp.Model("MSUC")

x = {}
y = {}
s_up = {}
s_down = {}
ys_p = {}
ys_n = {}

# TODO Set startup and shutdown costs
# suc = [10]*n_gens
# sdc = [10]*n_gens

for t in range(n_stages):
    for n in range(n_nodes_per_stage[t]):
        for g in range(n_gens):
            x[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"x_{t+1}_{n+1}_{g+1}")
            y[t,n,g] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"y_{t+1}_{n+1}_{g+1}")
            s_up[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"s_up_{t+1}_{n+1}_{g+1}")
            s_down[t,n,g] = model.addVar(vtype = gp.GRB.BINARY, name = f"s_down_{t+1}_{n+1}_{g+1}")
        ys_p[t,n] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_p_{t+1}_{n+1}")
        ys_n[t,n] = model.addVar(vtype = gp.GRB.CONTINUOUS, lb = 0, name = f"ys_n_{t+1}_{n+1}")

model.update()

### Constraints

In [113]:
# Objective
obj = gp.quicksum(prob[t][n]*(gc[g]*y[t,n,g] + suc[g]*s_up[t,n,g] + sdc[g]*s_down[t,n,g] + penalty*(ys_p[t,n] + ys_n[t,n])) 
                    for t in range(n_stages)
                    for n in range(n_nodes_per_stage[t])
                    for g in range(n_gens))

model.setObjective(obj)


# Balance constraints
model.addConstrs((gp.quicksum(y[t,n,g] for g in range(n_gens)) + ys_p[t,n] - ys_n[t,n] == gp.quicksum(p_d[t][n])
                    for t in range(n_stages)
                    for n in range(n_nodes_per_stage[t])),                    
                    "balance")


# Generator constraints
model.addConstrs((y[t,n,g] >= pg_min[g]*x[t,n,g] 
                    for g in range(n_gens)
                    for t in range(n_stages) 
                    for n in range(n_nodes_per_stage[t])),
                    "min-generation")

model.addConstrs((y[t,n,g] <= pg_max[g]*x[t,n,g]  
                    for g in range(n_gens)
                    for t in range(n_stages) 
                    for n in range(n_nodes_per_stage[t])),
                    "max-generation")


# Power flow constraints
for t in range(n_stages):
    for n in range(n_nodes_per_stage[t]):
        line_flows = [gp.quicksum(ptdf[l,b] * (gp.quicksum(y[t,n,g] for g in gens_at_bus[b]) - p_d[t][n][b])
                        for b in range(n_buses)) for l in range(n_lines)]
        model.addConstrs((line_flows[l] <= pl_max[l] for l in range(n_lines)), "power-flow(1)")
        model.addConstrs((-line_flows[l] <= pl_max[l] for l in range(n_lines)), "power-flow(2)")


# Startup shutdown constraints
# t=0
x_init = [0]*n_gens
model.addConstrs((x[0,0,g] - x_init[g] <= s_up[0,0,g] for g in range(n_gens)), "up-down(1)")
model.addConstrs((x[0,0,g] - x_init[g] == s_up[0,0,g] - s_down[0,0,g] for g in range(n_gens)), "up-down(2)")
# t>0
for t in range(1,n_stages):
    for n in range(n_nodes_per_stage[t]):
        model.addConstrs((x[t,n,g] - x[t-1,n_prev,g] <= s_up[t,n,g] 
                        for g in range(n_gens) 
                        for n_prev in range(n_nodes_per_stage[t-1])), 
                        "up-down(1)")
        model.addConstrs((x[t,n,g] - x[t-1,n_prev,g] <= s_up[t,n,g] - s_down[t,n,g] 
                        for g in range(n_gens)
                        for n_prev in range(n_nodes_per_stage[t-1])), 
                        "up-down(2)")


# Ramp rate constraints
# t=0
y_init = [0]*n_gens
model.addConstrs((y[0,0,g] - y_init[g] <= rg_up_max[g] for g in range(n_gens)), "rate-up")
model.addConstrs((y_init[g] - y[0,0,g] <= rg_down_max[g] for g in range(n_gens)), "rate-down(2)")
# t>0
for t in range(1,n_stages):
    for n in range(n_nodes_per_stage[t]):
        model.addConstrs((y[t,n,g] - y[t-1,n_prev,g] <= rg_up_max[g]
                        for g in range(n_gens)
                        for n_prev in range(n_nodes_per_stage[t-1])), 
                        "rate-up")
        model.addConstrs((y[t-1,n_prev,g] - y[t,n,g] <= rg_down_max[g]
                        for g in range(n_gens)
                        for n_prev in range(n_nodes_per_stage[t-1])), 
                        "rate-down")


# Minimum up- and down-time constraints
for g in range(n_gens):
    for t in range(1, min_up_time[g]):
        for n in range(n_nodes_per_stage[t]):
            previous_nodes_list = [list(range(n_nodes_per_stage[_t])) for _t in range(t)]
            previous_nodes_perm = list(product(*previous_nodes_list))
            model.addConstrs((gp.quicksum(x[_t,_n,g] for _t,_n in zip(range(t), perm)) 
                    >= (t+1)*s_down[t,n,g] for perm in previous_nodes_perm), "min-uptime")
    
    for t in range(min_up_time[g], n_stages):
        for n in range(n_nodes_per_stage[t]):
            previous_nodes_list = [list(range(n_nodes_per_stage[_t])) for _t in range(t-min_up_time[g],t)]
            previous_nodes_perm = list(product(*previous_nodes_list))
            model.addConstrs((gp.quicksum(x[_t,_n,g] for _t,_n in zip(range(t-min_up_time[g],t), perm)) 
                    >= min_up_time[g]*s_down[t,n,g] for perm in previous_nodes_perm), "min-uptime")

    for t in range(1, min_down_time[g]):
        for n in range(n_nodes_per_stage[t]):
            previous_nodes_list = [list(range(n_nodes_per_stage[_t])) for _t in range(t)]
            previous_nodes_perm = list(product(*previous_nodes_list))
            model.addConstrs((gp.quicksum((1-x[_t,_n,g]) for _t,_n in zip(range(t), perm)) 
                    >= (t+1)*s_up[t,n,g] for perm in previous_nodes_perm), "min-downtime")

    for t in range(min_down_time[g], n_stages):
        for n in range(n_nodes_per_stage[t]):
            previous_nodes_list = [list(range(n_nodes_per_stage[_t])) for _t in range(t-min_down_time[g],t)]
            previous_nodes_perm = list(product(*previous_nodes_list))
            model.addConstrs((gp.quicksum((1-x[_t,_n,g]) for _t,_n in zip(range(t-min_down_time[g],t), perm)) 
                    >= min_down_time[g]*s_up[t,n,g] for perm in previous_nodes_perm), "min-downtime")

                 


model.update()
#model.display()

### Solve

In [114]:
#model.setParam("OutputFlag",0)

model.optimize()

#model.setParam("OutputFlag",1)

model.printAttr("X")
print()
print(f"Optimal value: {obj.getValue()}")

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 426 rows, 96 columns and 1084 nonzeros
Model fingerprint: 0xeab82381
Variable types: 48 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [7e-01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 1.699409e+07
Presolve removed 297 rows and 68 columns
Presolve time: 0.00s
Presolved: 129 rows, 28 columns, 384 nonzeros
Variable types: 0 continuous, 28 integer (28 binary)
Found heuristic solution: objective 3398.8179691

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 16 (of 16 available processors)

Solution count 2: 3398.82 1.69941e+07 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.398817969121e+03, best bound 3.398817969121e+03, gap 0.0000%

    Variable   

In [115]:
x_out = [f"x[{t+1},{n+1},{g+1}]:  {x[t,n,g].x}" for t in range(n_stages) for n in range(n_nodes_per_stage[t]) for g in range(n_gens)]
y_out = [f"y[{t+1},{n+1},{g+1}]:  {y[t,n,g].x}" for t in range(n_stages) for n in range(n_nodes_per_stage[t]) for g in range(n_gens)]
s_up_out = [f"s_up[{t+1},{n+1},{g+1}]:  {s_up[t,n,g].x}" for t in range(n_stages) for n in range(n_nodes_per_stage[t]) for g in range(n_gens)]
s_down_out = [f"s_down[{t+1},{n+1},{g+1}]:  {s_down[t,n,g].x}" for t in range(n_stages) for n in range(n_nodes_per_stage[t]) for g in range(n_gens)]

print("Commitment decisions")
for text in x_out:
    print(f"{text}")

print("Dispatch decisions")
for text in y_out:
    print(f"{text}")

print("Startup decisions")
for text in s_up_out:
    print(f"{text}")

print("Shutdown decisions")
for text in s_down_out:
    print(f"{text}")

Commitment decisions
x[1,1,1]:  1.0
x[2,1,1]:  1.0
x[2,2,1]:  1.0
x[2,3,1]:  1.0
x[3,1,1]:  1.0
x[3,2,1]:  1.0
x[3,3,1]:  1.0
x[4,1,1]:  1.0
x[4,2,1]:  1.0
x[4,3,1]:  1.0
x[5,1,1]:  1.0
x[5,2,1]:  1.0
x[5,3,1]:  1.0
x[6,1,1]:  1.0
x[6,2,1]:  1.0
x[6,3,1]:  1.0
Dispatch decisions
y[1,1,1]:  139.0873527447768
y[2,1,1]:  195.1466041721651
y[2,2,1]:  210.21840137903416
y[2,3,1]:  206.90032610348803
y[3,1,1]:  363.9174025522841
y[3,2,1]:  372.5431424286558
y[3,3,1]:  308.8405668452222
y[4,1,1]:  317.0460774313105
y[4,2,1]:  359.4151257183172
y[4,3,1]:  350.5529031929165
y[5,1,1]:  339.3845978470828
y[5,2,1]:  318.15416626996307
y[5,3,1]:  306.18754783684443
y[6,1,1]:  343.02886192491786
y[6,2,1]:  337.36586140316064
y[6,3,1]:  352.2633103414949
Startup decisions
s_up[1,1,1]:  1.0
s_up[2,1,1]:  0.0
s_up[2,2,1]:  0.0
s_up[2,3,1]:  0.0
s_up[3,1,1]:  0.0
s_up[3,2,1]:  0.0
s_up[3,3,1]:  0.0
s_up[4,1,1]:  0.0
s_up[4,2,1]:  0.0
s_up[4,3,1]:  0.0
s_up[5,1,1]:  0.0
s_up[5,2,1]:  0.0
s_up[5,3,1]:  0.

In [116]:
costs = []
c = 0
for t in reversed(range(n_stages)):
    for n in range(n_nodes_per_stage[t]):
        for g in range(n_gens):
            c += prob[t][n] * y[t,n,g].x*gc[g]
    costs.append(c)


costs

[688.4386891130489,
 1330.922897082309,
 2015.5989679773384,
 2712.4663758614465,
 3120.643263631238,
 3398.8179691207915]

In [117]:
model.display()

Minimize
<gurobi.LinExpr: 2.0 y_1_1_1 + 10000.0 ys_p_1_1 + 10000.0 ys_n_1_1
+ 0.6666666666666666 y_2_1_1 + 3333.333333333333 ys_p_2_1 + 3333.333333333333 ys_n_2_1
+ 0.6666666666666666 y_2_2_1 + 3333.333333333333 ys_p_2_2 + 3333.333333333333 ys_n_2_2
+ 0.6666666666666666 y_2_3_1 + 3333.333333333333 ys_p_2_3 + 3333.333333333333 ys_n_2_3
+ 0.6666666666666666 y_3_1_1 + 3333.333333333333 ys_p_3_1 + 3333.333333333333 ys_n_3_1
+ 0.6666666666666666 y_3_2_1 + 3333.333333333333 ys_p_3_2 + 3333.333333333333 ys_n_3_2
+ 0.6666666666666666 y_3_3_1 + 3333.333333333333 ys_p_3_3 + 3333.333333333333 ys_n_3_3
+ 0.6666666666666666 y_4_1_1 + 3333.333333333333 ys_p_4_1 + 3333.333333333333 ys_n_4_1
+ 0.6666666666666666 y_4_2_1 + 3333.333333333333 ys_p_4_2 + 3333.333333333333 ys_n_4_2
+ 0.6666666666666666 y_4_3_1 + 3333.333333333333 ys_p_4_3 + 3333.333333333333 ys_n_4_3
+ 0.6666666666666666 y_5_1_1 + 3333.333333333333 ys_p_5_1 + 3333.333333333333 ys_n_5_1
+ 0.6666666666666666 y_5_2_1 + 3333.333333333333 ys_p_